Below is code with a link to a happy or sad dataset which contains 80 images, 40 happy and 40 sad. 
Create a convolutional neural network that trains to 100% accuracy on these images,  which cancels training upon hitting training accuracy of >.999

Hint -- it will work best with 3 convolutional layers.

In [36]:
import os
import zipfile


DESIRED_ACCURACY = 0.999

!wget --no-check-certificate \
    "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip" \
    -O "/tmp/happy-or-sad.zip"

zip_ref = zipfile.ZipFile("/tmp/happy-or-sad.zip", 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()



--2019-03-14 14:01:24--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.188.128, 2404:6800:4008:c00::80
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.188.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2670333 (2.5M) [application/zip]
Saving to: ‘/tmp/happy-or-sad.zip’

/tmp/happy-or-sad.z 100%[===================>]   2.55M  --.-KB/s    in 0.02s   

2019-03-14 14:01:24 (117 MB/s) - ‘/tmp/happy-or-sad.zip’ saved [2670333/2670333]



In [0]:
# Directory with our training horse pictures
train_h_dir = os.path.join('/tmp/h-or-s/happy')

# Directory with our training human pictures
train_s_dir = os.path.join('/tmp/h-or-s/sad')

In [38]:
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
print(tf.__version__)

1.13.1


In [0]:
# Set up the Callback
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('acc')>DESIRED_ACCURACY):
            print("\n Reached 99.8% accuracy so cancelling training!")
            self.model.stop_training = True
            
callbacks = myCallback()

In [40]:
model = tf.keras.models.Sequential([
   
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
   
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
  
    tf.keras.layers.Flatten(),
    
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 148, 148, 64)      1792      
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 74, 74, 64)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 72, 72, 32)        18464     
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 34, 34, 32)        9248      
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 17, 17, 32)        0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 9248)              0         
__________

In [0]:
model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['acc'])

In [42]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1/255)

# Flow training images 
train_generator = train_datagen.flow_from_directory(
        '/tmp/h-or-s/',
        batch_size=20,
        target_size=(150, 150),  
        class_mode='binary')


Found 80 images belonging to 2 classes.


In [43]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=2,  
      epochs=50,
      verbose=1,
      callbacks=[callbacks])

Epoch 1/50
4/4 [==============================] - 0s 111ms/step - loss: 3.1815 - acc: 0.4250
Epoch 2/50
4/4 [==============================] - 0s 71ms/step - loss: 0.7562 - acc: 0.6125
Epoch 3/50
4/4 [==============================] - 0s 45ms/step - loss: 0.5984 - acc: 0.7375
Epoch 4/50
4/4 [==============================] - 0s 68ms/step - loss: 0.5599 - acc: 0.6500
Epoch 5/50
4/4 [==============================] - 0s 50ms/step - loss: 0.5282 - acc: 0.7375
Epoch 6/50
4/4 [==============================] - 0s 81ms/step - loss: 0.2213 - acc: 0.9375
Epoch 7/50
4/4 [==============================] - 0s 52ms/step - loss: 0.1814 - acc: 0.9500
Epoch 8/50
4/4 [==============================] - 0s 67ms/step - loss: 0.2935 - acc: 0.8375
Epoch 9/50
4/4 [==============================] - 0s 54ms/step - loss: 0.1881 - acc: 0.8750
Epoch 10/50
4/4 [==============================] - 0s 93ms/step - loss: 0.3015 - acc: 0.8750
Epoch 11/50
4/4 [==============================] - 0s 54ms/step - loss: 0.0998